# 模型选择，欠拟合和过拟合
    当模型在训练数据集上更准确时，它在测试数据集上却不一定更准确。
## 一.训练误差和泛化误差
    训练误差（training error）：指模型在训练数据集上表现出的误差
    泛化误差（generalization error）：指模型在任意一个测试数据样本上表现出的误差的期望，并常常通过测试数据集上的误差来近似。
    
    计算训练误差和泛化误差可以使用之前介绍过的损失函数，例如线性回归用到的平方损失函数和 softmax 回归用到的交叉熵损失函数。
    
    在机器学习里，我们通常假设训练数据集（训练题）和测试数据集（测试题）里的每一个样本都是从同一个概率分布中相互独立地生成的。
    基于该独立同分布假设，给定任意一个机器学习模型（含参数），它的训练误差的期望和泛化误差都是一样的。
    例如我们将模型参数设成随机值（小学生），那么训练误差和泛化误差会非常相近。
    然而我们从之前的章节中已经了解到，模型的参数是通过在训练数据集上训练模型而学习出的，参数的选择依据了最小化训练误差（高三备考生）。
    所以，训练误差的期望小于或等于泛化误差。也就是说，一般情况下，由训练数据集学到的模型参数会使模型在训练数据集上的表现优于或等于在测试数据集上的表现。
    由于无法从训练误差估计泛化误差，一味地降低训练误差并不意味着泛化误差一定会降低。

    机器学习模型应关注降低泛化误差。
## 二.模型选择（model selection）
    可供选择的候选模型可以是有着不同超参数的同类模型。
### 2.1 验证数据集（validation dataset）
    严格意义上，测试集只能在所有超参数和模型参数选定后使用一次。
    我们不可以使用测试数据选择模型，例如调参。
    由于我们无法从训练误差估计泛化误差，因此也不应只依赖训练数据选择模型。
    有鉴于此，我们可以预留一部分在训练数据集和测试数据集以外的数据来进行模型选择。
    这部分数据被称为验证数据集，简称验证集（validation set）。例如，我们可以从给定的训练集中随机选取一小部分作为验证集，而将剩余部分作为真正的训练集。

    在实际应用中，由于数据不容易获取，测试数据极少只使用一次就丢弃。
    因此，实践中验证数据集和测试数据集的界限可能比较模糊。
    严格意义上，除非明确说明，实验所使用的测试集应为验证集，实验报告的测试精度应为验证精度。
### 2.2 K 折交叉验证（K-fold cross-validation）-- 训练数据不够用时的改进方法
    在 K 折交叉验证中，我们把原始训练数据集分割成 K 个不重合的子数据集，然后做 K 次模型训练和验证。
    每一次，我们使用一个子数据集验证模型，并使用其他 K−1 个子数据集来训练模型。
    在这 K 次训练和验证中，每次用来验证模型的子数据集都不同。
    最后，我们对这 K 次训练误差和验证误差分别求平均。
## 三.欠拟合和过拟合
    模型无法得到较低的训练误差，我们将这一现象称作欠拟合（underfitting）
    模型的训练误差远小于它在测试数据集上的误差，我们称该现象为过拟合（overfitting）。
    造成因素：模型复杂度和训练数据集
### 3.1 模型复杂度
    由于高阶多项式函数模型参数更多，模型函数的选择空间更大，所以高阶多项式函数比低阶多项式函数的复杂度更高。
    因此，高阶多项式函数比低阶多项式函数更容易在相同的训练数据集上得到更低的训练误差。
    给定训练数据集，如果模型的复杂度过低，很容易出现欠拟合；如果模型复杂度过高，很容易出现过拟合。
![模型复杂度对欠拟合和过拟合的影响](../img/capacity_vs_error.svg)
### 3.2 训练数据集大小
    一般来说，如果训练数据集中样本数过少，特别是比模型参数数量（按元素计）更少时，过拟合更容易发生。
    此外，泛化误差不会随训练数据集里样本数量增加而增大。
    因此，在计算资源允许范围之内，我们通常希望训练数据集大一些，特别当模型复杂度较高时：例如层数较多的深度学习模型。
## 四.多项式函数拟合实验

In [1]:
%matplotlib inline
import gluonbook as gbook
from mxnet import autograd,gluon,nd
from mxnet.gluon import data as gdata,loss as gloss,nn

### 4.1 生成数据集
$$y=1.2x+3.4x^2+5.6x^3+5+\epsilon,$$

In [3]:
n_train,n_test,true_w,true_b=100,100,[1.2,-3.4,5.6],5
features=nd.random.normal(shape=(n_train+n_test,1))
ploy_features=nd.concat(features,nd.power(features,2),nd.power(features,3))
labels=(true_w[0]*ploy_features[:,0]+true_w[1]*ploy_features[:,1]+true_w[2]*ploy_features[:,2]+true_b)
labels+=nd.random.normal(scale=0.1,shape=labels.shape)

In [4]:
features[:2],ploy_features[:2],labels[:2]

(
 [[ 1.16307867]
  [ 0.48380461]]
 <NDArray 2x1 @cpu(0)>, 
 [[ 1.16307867  1.35275197  1.57335699]
  [ 0.48380461  0.2340669   0.11324265]]
 <NDArray 2x3 @cpu(0)>, 
 [ 10.5346508    5.53009319]
 <NDArray 2 @cpu(0)>)

### 4.2 定义、训练和测试模型

In [5]:
# 本函数已保存在 gluonbook 包中方便以后使用。
# 作图函数semilogy，其中y轴使用了对数尺度g
def semilogy(x_vals,y_vals,x_label,y_label,x2_vals=None,y2_vals=None,legend=None,figsize=(3.5,2.5)):
    gb.set_figsize(figsize)
    gb.plt_xlabel(x_label)
    gb.plt_ylabel(y_label)
    gb.plt.semilogy(x_vals,y_vals)
    if x2_vals and y2_vals:
        gb.plt.semilogy(x2_vals,y2_vals,linestyle=':')
        gb.plt.lengend(legend)